In [1]:
!conda activate DS807

'conda' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import data_loader as dl
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn import ensemble

In [3]:
df = dl.load_1d_grays()
df

,index,string,CC,D,Y,gray_value,filename
0,1,1836,0,3,6,"[[[166.0], [167.0], [166.0], [162.0], [164.0],...",1.jpg
1,2,1836,0,3,6,"[[[169.0], [168.0], [169.0], [170.0], [166.0],...",2.jpg
2,3,1840,0,4,0,"[[[167.0], [169.0], [169.0], [166.0], [165.0],...",3.jpg
3,4,1840,0,4,0,"[[[167.0], [165.0], [165.0], [165.0], [163.0],...",4.jpg
4,5,1823,0,2,3,"[[[62.0], [45.0], [45.0], [37.0], [40.0], [33....",5.jpg
...,...,...,...,...,...,...,...
11995,11996,1808,0,0,8,"[[[88.0], [62.0], [60.0], [68.0], [61.0], [101...",11996.jpg
11996,11997,1809,0,0,9,"[[[173.0], [171.0], [173.0], [175.0], [174.0],...",11997.jpg
11997,11998,1840,0,4,0,"[[[189.0], [189.0], [189.0], [189.0], [189.0],...",11998.jpg
11998,11999,1840,0,4,0,"[[[191.0], [192.0], [192.0], [192.0], [193.0],...",11999.jpg


In [4]:
df['gray_value'][0].shape

(56, 106, 1)

In [5]:

arr = np.zeros(shape=(12000,56,106))
i = 0
while i < 12000:
    arr[i] = np.squeeze(df['gray_value'][i])
    i += 1
arr.shape


(12000, 56, 106)

In [6]:
n_samples = len(arr)
x_values = arr.reshape((n_samples, -1))
x_values.shape

(12000, 5936)

In [7]:
from sklearn.model_selection import train_test_split
y_values = df['D']
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.2, random_state=2022)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2022)
print(x_train.shape, x_val.shape, x_test.shape)


(7680, 5936) (1920, 5936) (2400, 5936)


In [24]:

results = []
n_iterations = [20,50,100,200]#,200,500,800]
lr = [0.01,0.1]#,0.001,0.1]
n_depth = [2,4]#,6,8,10]
loss_f = 'MultiClass'

for iters in n_iterations:
    for ls in lr:
        for depths in n_depth:
            model = CatBoostClassifier(iterations=iters,
                                        learning_rate=ls,
                                        depth=depths,
                                        loss_function=loss_f)

            model.fit(x_train, y_train, verbose=10)
            acc = model.score(x_val,y_val)
            # the results from the different hyperparameters are then stored so we can get the best one
            results.append([acc,ls,iters, depths])
results_df = pd.DataFrame(results)
results_df.columns=['accuracy',"learning_rate","n_estimators", "depth"]
#prints the results from best to worst in regards to accuracy, listting the hyperparameters for the result
results_df = results_df.sort_values('accuracy', ascending=False)
results_df

0:	learn: 2.1870465	total: 368ms	remaining: 7s
10:	learn: 2.0958620	total: 3.13s	remaining: 2.56s
19:	learn: 2.0283492	total: 5.81s	remaining: 0us
0:	learn: 2.1870936	total: 1.15s	remaining: 21.8s
10:	learn: 2.0956735	total: 9.42s	remaining: 7.71s
19:	learn: 2.0274402	total: 17.2s	remaining: 0us
0:	learn: 2.1000270	total: 366ms	remaining: 6.96s
10:	learn: 1.6935304	total: 3.46s	remaining: 2.83s
19:	learn: 1.5886845	total: 6.56s	remaining: 0us
0:	learn: 2.1003785	total: 1.02s	remaining: 19.4s
10:	learn: 1.6820533	total: 9.34s	remaining: 7.64s
19:	learn: 1.5638210	total: 17.4s	remaining: 0us
0:	learn: 2.1870465	total: 394ms	remaining: 19.3s
10:	learn: 2.0958620	total: 3.68s	remaining: 13s
20:	learn: 2.0215546	total: 7.13s	remaining: 9.84s
30:	learn: 1.9596479	total: 10.6s	remaining: 6.51s
40:	learn: 1.9074476	total: 14.3s	remaining: 3.15s
49:	learn: 1.8673470	total: 17.7s	remaining: 0us
0:	learn: 2.1870936	total: 1.24s	remaining: 1m
10:	learn: 2.0956735	total: 10.1s	remaining: 35.7s
20:	

,accuracy,learning_rate,n_estimators,depth
15,0.525000,0.10,200,4
11,0.452604,0.10,100,4
14,0.451042,0.10,200,2
10,0.398438,0.10,100,2
7,0.386458,0.10,50,4
6,0.364583,0.10,50,2
13,0.329688,0.01,200,4
3,0.328646,0.10,20,4
12,0.305729,0.01,200,2
2,0.297396,0.10,20,2


In [25]:
results_df.to_csv('Catboost_results')